In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Atherosclerosis"
cohort = "GSE123088"

# Input paths
in_trait_dir = "../../input/GEO/Atherosclerosis"
in_cohort_dir = "../../input/GEO/Atherosclerosis/GSE123088"

# Output paths
out_data_file = "../../output/preprocess/Atherosclerosis/GSE123088.csv"
out_gene_data_file = "../../output/preprocess/Atherosclerosis/gene_data/GSE123088.csv"
out_clinical_data_file = "../../output/preprocess/Atherosclerosis/clinical_data/GSE123088.csv"
json_path = "../../output/preprocess/Atherosclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A validated single-cell-based strategy to identify diagnostic and therapeutic targets in complex diseases"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell type: CD4+ T cells'], 1: ['primary diagnosis: ASTHMA', 'primary diagnosis: ATHEROSCLEROSIS', 'primary diagnosis: BREAST_CANCER', 'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA', 'primary diagnosis: CROHN_DISEASE', 'primary diagnosis: ATOPIC_ECZEMA', 'primary diagnosis: HEALTHY_CONTROL', 'primary diagnosis: INFLUENZA', 'primary diagnosis: OBESITY', 'primary diagnosis: PSORIASIS', 'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS', 'primary diagnosis: TYPE_1_DIABETES', 'primary diagnosis: ACUTE_TONSILLITIS', 'primary diagnosis: ULCERATIVE_COLITIS', 'primary diagnosis: Breast cancer', 'primary diagnosis: Control'], 2: ['Sex: Male', 'diagnosis2: ATOPIC_ECZEMA', 'Sex: Female'

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Determine gene expression data availability
# Based on the background information, this dataset appears to be a SuperSeries containing gene expression data
is_gene_available = True

# 2. Identify variable availability and create conversion functions

# 2.1 For trait (Atherosclerosis)
trait_row = 1  # "primary diagnosis" in row 1 contains the trait information

def convert_trait(value: str) -> int:
    """Convert trait value to binary (0 or 1)."""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary based on Atherosclerosis diagnosis
    if "ATHEROSCLEROSIS" in value.upper():
        return 1
    elif "HEALTHY_CONTROL" in value.upper() or "CONTROL" in value.upper():
        return 0
    else:
        # Other diagnoses are not related to Atherosclerosis
        return 0

# 2.2 For age
age_row = 3  # "age" appears in row 3 and 4, but primarily in row 3

def convert_age(value: str) -> float:
    """Convert age value to continuous numeric value."""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

# 2.3 For gender
gender_row = 2  # "Sex" appears in row 2 and 3, but primarily in row 2

def convert_gender(value: str) -> int:
    """Convert gender value to binary (0 for female, 1 for male)."""
    if pd.isna(value):
        return None
    
    # Extract the value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip().upper()
    
    if "FEMALE" in value:
        return 0
    elif "MALE" in value:
        return 1
    else:
        return None

# 3. Save metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    try:
        # Use the clinical_data variable that should be available from a previous step
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the selected clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the selected clinical features to a CSV file
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to: {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        # If clinical data is not available or there's an error, we'll proceed without it
        print("Proceeding without clinical data.")


Preview of selected clinical features:
{'GSM3494884': [0.0, 56.0, 1.0], 'GSM3494885': [0.0, nan, nan], 'GSM3494886': [0.0, 20.0, 0.0], 'GSM3494887': [0.0, 51.0, 0.0], 'GSM3494888': [0.0, 37.0, 1.0], 'GSM3494889': [0.0, 61.0, 1.0], 'GSM3494890': [0.0, nan, nan], 'GSM3494891': [0.0, 31.0, 1.0], 'GSM3494892': [0.0, 56.0, 0.0], 'GSM3494893': [0.0, 41.0, 0.0], 'GSM3494894': [0.0, 61.0, 0.0], 'GSM3494895': [1.0, nan, nan], 'GSM3494896': [1.0, 80.0, 1.0], 'GSM3494897': [1.0, 53.0, 1.0], 'GSM3494898': [1.0, 61.0, 1.0], 'GSM3494899': [1.0, 73.0, 1.0], 'GSM3494900': [1.0, 60.0, 1.0], 'GSM3494901': [1.0, 76.0, 1.0], 'GSM3494902': [1.0, 77.0, 0.0], 'GSM3494903': [1.0, 74.0, 0.0], 'GSM3494904': [1.0, 69.0, 1.0], 'GSM3494905': [0.0, 77.0, 0.0], 'GSM3494906': [0.0, 81.0, 0.0], 'GSM3494907': [0.0, 70.0, 0.0], 'GSM3494908': [0.0, 82.0, 0.0], 'GSM3494909': [0.0, 69.0, 0.0], 'GSM3494910': [0.0, 82.0, 0.0], 'GSM3494911': [0.0, 67.0, 0.0], 'GSM3494912': [0.0, 67.0, 0.0], 'GSM3494913': [0.0, 78.0, 0.0], 'GS

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Atherosclerosis/GSE123088/GSE123088_series_matrix.txt.gz


Gene data shape: (24166, 204)
First 20 gene/probe identifiers:
Index(['1', '2', '3', '9', '10', '12', '13', '14', '15', '16', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the previous step
# These appear to be simple numeric identifiers (1, 2, 3, etc.), not human gene symbols
# These are likely probe IDs or some other type of numeric identifiers that need mapping to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Based on the preview, we can see that we have ID and ENTREZ_GENE_ID
# We need to check if there's any other information in the SOFT file that could help us map to gene symbols

# Check the first few rows of the SOFT file to better understand its structure
print("\nChecking the SOFT file structure:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if i < 20:  # Just check the first 20 lines
            print(line.strip())
        else:
            break

print("\nLet's create a mapping using ENTREZ_GENE_ID as this corresponds to gene identifiers in NCBI")
# Create a mapping dataframe with probe IDs and ENTREZ_GENE_ID (which are numeric gene identifiers)
mapping_data = gene_annotation[['ID', 'ENTREZ_GENE_ID']].copy()
mapping_data = mapping_data.dropna(subset=['ENTREZ_GENE_ID'])

# Since ENTREZ_GENE_ID is already in the format of gene identifiers, we'll use those directly
mapping_data = mapping_data.rename(columns={'ENTREZ_GENE_ID': 'Gene'})

# Filter out rows with empty gene values
mapping_data = mapping_data[mapping_data['Gene'] != '']
print(f"Mapping data shape: {mapping_data.shape}")
print("Sample mapping data:")
print(preview_df(mapping_data, n=5))

# Count how many probes map to at least one gene identifier
genes_mapped = mapping_data.shape[0]
total_probes = gene_annotation.shape[0]
mapping_percentage = (genes_mapped / total_probes) * 100 if total_probes > 0 else 0
print(f"\nFound gene identifiers for {genes_mapped} out of {total_probes} probes ({mapping_percentage:.2f}%)")

# Save the mapping for later use
gene_mapping = mapping_data



Gene annotation preview:
Columns in gene annotation: ['ID', 'ENTREZ_GENE_ID', 'SPOT_ID']
{'ID': ['1', '2', '3', '9', '10'], 'ENTREZ_GENE_ID': ['1', '2', '3', '9', '10'], 'SPOT_ID': [1.0, 2.0, 3.0, 9.0, 10.0]}

Checking the SOFT file structure:
^DATABASE = GeoMiame
!Database_name = Gene Expression Omnibus (GEO)
!Database_institute = NCBI NLM NIH
!Database_web_link = http://www.ncbi.nlm.nih.gov/geo
!Database_email = geo@ncbi.nlm.nih.gov
^SERIES = GSE123088
!Series_title = A validated single-cell-based strategy to identify diagnostic and therapeutic targets in complex diseases
!Series_geo_accession = GSE123088
!Series_status = Public on Nov 23 2021
!Series_submission_date = Nov 28 2018
!Series_last_update_date = Apr 21 2023
!Series_pubmed_id = 31358043
!Series_summary = This SuperSeries is composed of the SubSeries listed below.
!Series_overall_design = Refer to individual Series
!Series_type = Expression profiling by array
!Series_sample_id = GSM3494884
!Series_sample_id = GSM3494885
!S

Mapping data shape: (4740924, 2)
Sample mapping data:
{'ID': ['1', '2', '3', '9', '10'], 'Gene': ['1', '2', '3', '9', '10']}

Found gene identifiers for 4740924 out of 4740924 probes (100.00%)


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the gene identifiers in the gene expression data and the gene annotation data:
# - Gene expression data identifiers: numeric IDs like '1', '2', '3', etc. (probe IDs)
# - Gene annotation contains 'ID', 'ENTREZ_GENE_ID', 'SPOT_ID' columns
# - 'ID' in gene_annotation matches the index of gene_data (probe IDs)
# - 'ENTREZ_GENE_ID' contains Entrez Gene IDs which can be mapped to human gene symbols

# 2. Create a gene mapping dataframe 
# We need 'ID' as the identifier and 'ENTREZ_GENE_ID' as the gene reference
mapping_df = gene_annotation[['ID', 'ENTREZ_GENE_ID']].copy()
mapping_df = mapping_df.rename(columns={'ENTREZ_GENE_ID': 'Gene'})
mapping_df = mapping_df.dropna(subset=['Gene'])

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# The function apply_gene_mapping handles the many-to-many relation between probes and genes
gene_data = apply_gene_mapping(gene_data, mapping_df)

print(f"Mapped gene expression data shape: {gene_data.shape}")
print("First 5 gene symbols in the mapped data:")
print(gene_data.index[:5].tolist())

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")

Mapped gene expression data shape: (0, 204)
First 5 gene symbols in the mapped data:
[]
Gene expression data saved to: ../../output/preprocess/Atherosclerosis/gene_data/GSE123088.csv
